In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../../data/'
group_dir_ = os.path.join('../../../HPCF/all/test_0.8/data/groups/low')
folder = 'all (user)'
file_name = 'normalized_to_rating_filter_track_5_user_100.pkl'

In [2]:
predictions = pd.read_pickle(os.path.join(dir_, folder, 'topN_pred_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
26,0,9,1.071429,15
37,0,15,0.214286,3
45,0,22,0.428571,6
64,0,28,0.285714,4
75,0,33,0.071429,1


In [3]:
all_user = pd.read_pickle(os.path.join('../../../HPCF/all/test_0.8/data/', file_name[:-3] + 'pkl'))
df = pd.read_pickle(os.path.join('../../../HPCF/all/test_0.8/data/', file_name[:-3] + 'pkl'))

In [4]:
predictions[:5]

,uid,tid,rating
0,0,9,0.432739
1,0,15,0.327982
2,0,22,0.266792
3,0,28,0.363382
4,0,33,0.345882


In [5]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
26,0,9,1.071429,15
37,0,15,0.214286,3
45,0,22,0.428571,6
64,0,28,0.285714,4
75,0,33,0.071429,1


In [6]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [7]:
num_user = len(test['uid'].unique())
num_user

953

In [8]:
pop_dict = {}
for i in all_user['tid'].unique():
    pop_dict[i] = len(all_user[all_user['tid']==i])/num_user

In [9]:
top_n_size = 50
top_n_lists_g = []
top_n_items_g = []
for groups in groups_n:

    group_size = len(groups[0])
    top_n_lists = []
    top_n_items = pd.DataFrame()

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 
        
        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)

        if len(top_n_items) == 0:
            top_n_items = rating_table
        else:
            top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()

    top_n_lists_g.append(top_n_lists)   
    top_n_items_g.append(top_n_items)

COV = []    
NOV = []
HitNOV = []
HitCOV = []
for i in range(len(groups_n)):
    groups = groups_n[i]
    top_n_lists = top_n_lists_g[i]
    top_n_items = top_n_items_g[i]

    nov = 0
    hitNov = 0
    hits = np.zeros(len(df['tid'].unique()), dtype=bool)
    for j in (range(len(groups))):
        group = groups[j]
        top_n_list = top_n_lists[j]

        for k in range(len(group)):
            for l in range(top_n_size):
                uid = group[k]
                tid = top_n_list[l]
                t = test[test['uid'] == uid]
                t = t[t['tid'] == tid]
                nov += (1-pop_dict[tid])
                if len(t) > 0:
                    hits[int(tid)] = True
                    hitNov += (1-pop_dict[tid])
    NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
    COV.append(len(top_n_items)/len(df['tid'].unique()))
    HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
    HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
print(NOV)
print(HitNOV)
print(COV)
print(HitCOV)

[0.6069086977121388, 0.6143404145795671, 0.612273558269597, 0.6056733123469837, 0.602332983560705, 0.5999644846234863, 0.59379616152729]
[0.07854031444657904, 0.05930529432598377, 0.04614267478716132, 0.03900855353111377, 0.03334171330301971, 0.028363409937386844, 0.02729902176488508]
[0.0015739336282343385, 0.001383538431270507, 0.0012375687802649031, 0.0010979456358247602, 0.0009900550242119225, 0.0010344805701701498, 0.0009646689979500784]
[0.0014660430166215006, 0.0013200733656158967, 0.0011614107014793708, 0.0010154410504737666, 0.000901203932295468, 0.0008885109191645459, 0.0008250458535099354]
